In [1]:

import numpy as np
import pandas as pd
import dill
from tqdm import tqdm
import os, sys
import statistics
from scipy.stats import iqr
import json
from datetime import datetime, date 
import time

from other.save_file_param_end_out import open_json
param_json = open_json()

from sklearn.metrics import f1_score
import mysql.connector

import warnings
warnings.filterwarnings('ignore')

In [2]:
# обращаю внимание - модель сохранена в dill, но с расширением .pkl

def predict_megafon():
    with open('other/models/model_dill.pkl', 'rb') as f:
        model = dill.load(f)

    data_test = pd.read_csv('data_test.csv')
    dict_w = list(data_test['id'].unique())
    con = mysql.connector.connect(host='bainel.ru', user='web_user', password='1111', db='megafon')
    sql = f"select * from features where id in {tuple(dict_w)}"
    data_features = pd.read_sql(sql, con)
    data_features.drop('index', axis=1, inplace=True)
    data = pd.merge_asof(data_test.sort_values('buy_time'), data_features.sort_values('buy_time'), 
                         on='buy_time', by='id', direction='nearest')
    y_pred = model.predict_proba(data)[:,1]
    data_test['target'] = y_pred
    data_test.to_csv('answers_test.csv', index=False)
    
    return pd.read_csv('answers_test.csv')

In [3]:
%time x_test = predict_megafon()

CPU times: user 27.5 s, sys: 4.01 s, total: 31.5 s
Wall time: 2min 15s


In [4]:
x_test.to_csv('answers_test.csv', index=False)
x_test

,Unnamed: 0,id,vas_id,buy_time,target
0,0,3130519,2.0,1548018000,0.115160
1,1,2000860,4.0,1548018000,0.123048
2,2,1099444,2.0,1546808400,0.849663
3,3,1343255,5.0,1547413200,0.869464
4,4,1277040,2.0,1546808400,0.856380
...,...,...,...,...,...
71226,71226,2502453,5.0,1548018000,0.850436
71227,71227,1693213,2.0,1548018000,0.120596
71228,71228,1891350,2.0,1548018000,0.120908
71229,71229,2437172,2.0,1548018000,0.889736
